In [6]:
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import time

# import functions for ODE parameter estimation
from model.diag_Gaussian import *

np.random.seed(123)

## Define ODE (x, time, parameters)

In [2]:
# ode 
def system(x, t, params): 

    # unpack parameters
    mu_max, pref, Sigma_inv, K_inv, delta, gamma = params
    
    # unpack variables
    od, pH = x 
    
    # growth rate modified by pH preference 
    # mu = mu_max * jnp.exp(-alpha * (pH - pref)**2)
           
    # coupled ODE for cells and environmental pH
    dod = od * (mu_max - K_inv * od) * (jnp.exp(-Sigma_inv * (pH - pref)**2.) - delta)
    dpH = (gamma * od) * (pH * (2. - pH / 7.))
    
    return jnp.array([dod, dpH])  

# define transform function to constrain parameter values
def transform(params):
    
    # unpack parameters
    mu_max, pref, Sigma_inv, K_inv, delta, gamma = params
    
    # take exp of strictly positive parameters
    mu_max = jnp.exp2(mu_max)
    pref = jnp.exp2(pref)*7.
    Sigma_inv = jnp.exp2(Sigma_inv)/100.
    K_inv = jnp.exp2(K_inv)/10.
    delta = jnp.exp2(delta)/10.
    gamma = gamma/100.
    
    return jnp.array([mu_max, pref, Sigma_inv, K_inv, delta, gamma])

# define prior mean of transformed variables 
prior_mean = jnp.array([1., 7., 1., 1., 0., 0.])

# prior precision 
alpha = 10. # jnp.array([10, 1, 100, 1, 1])

## Import data as a Pandas dataframe 

The dataframe must have a "Treatments" column that provides a unique identifier for each condition. The second column specifies the time point that observations were made. The following columns include the state variables.

In [3]:
# import data
df = pd.read_csv("Data/arc_allmono.csv").rename(columns={"Experiments":"Treatments"})

In [4]:
# species names
all_species = df.columns.values[3:]
all_species

array(['AC_OD', 'BA_OD', 'BC_OD', 'BL_OD', 'BT_OD', 'BV_OD', 'CC_OD',
       'DF_OD', 'wt_OD', 'delarc_OD', 'parc-_OD', 'parc+_OD'],
      dtype=object)

In [ ]:
# loop over monocultures
for sp in all_species:
    
    # init list of dataframes of model predictions
    df_preds = []

    # pH vals
    ph_vals = [4, 5, 6, 7, 8, 9]
    
    # loop over ph values
    for ph_val in ph_vals:
    
        # names of species in ODE
        species = [sp, 'pH']
    
        # pull out train and test dataframes
        df_train = pd.concat([df_exp for exp_name, df_exp in df.groupby("Treatments") if sp.split("_")[0] in exp_name and str(ph_val) not in exp_name])
        df_test  = pd.concat([df_exp for exp_name, df_exp in df.groupby("Treatments") if sp.split("_")[0] in exp_name and str(ph_val) in exp_name])
        
        # average test data over time
        df_groups = df_test.groupby("Time")
        df_test = df_groups[species].mean().reset_index()
        df_test.insert(0, "Treatments", [sp.split("_")[0]+"_"+str(ph_val)]*df_test.shape[0])
        
        # instantiate gLV fit 
        ode_model = ODE(system = system, 
                    transform = transform,
                    dataframe = df_train, 
                    sys_vars = species, 
                    prior_mean = prior_mean, 
                    alpha=alpha, beta=10.)

        # fit to data 
        ode_model.fit_posterior_EM(n_sample_sgd=1, 
                                   n_sample_hypers=50, 
                                   n_sample_evidence=100, 
                                   patience=3)


        # predict values and save to test dataframe
        x0 = df_test[species].values[0]
        y_preds = ode_model.predict_sample(x0, np.array(df_test.Time.values, float), n_sample=100)
        
        # save predictions 
        df_test.drop(columns='pH', inplace=True)   
        df_test[[f'sample_{i+1}' for i in range(y_preds.shape[0])]] = y_preds[:, :, 0].T
        
        # append to list of dataframes
        df_preds.append(df_test)
        
    # save dataframe
    df_pred = pd.concat(df_preds)
    df_pred.to_csv("kfold/{:}.csv".format(sp.split("_")[0]), index=False)

Computing model evidence...


100%|████████████████████████████████████████| 100/100 [00:00<00:00, 249.65it/s]


Log evidence: -1131.344
Updating hyperparameters...


100%|██████████████████████████████████████████| 50/50 [00:00<00:00, 224.51it/s]
/Users/Cissy/Desktop/Grad_School/Lab Work/Lab Note and Protocols/arc operon project/Liu_et_al_2024/model/diag_Gaussian.py:514: RuntimeWarning: overflow encountered in square
  v = beta2 * v + (1 - beta2) * (gradient ** 2)


Updating posterior...
Epoch 1, NEG ELBO: 106.712
Epoch 2, NEG ELBO: 59.046
Epoch 3, NEG ELBO: 52.101
Epoch 4, NEG ELBO: 47.065
Epoch 5, NEG ELBO: 43.774
Epoch 6, NEG ELBO: 40.380
Epoch 7, NEG ELBO: 37.495
Epoch 8, NEG ELBO: 34.018
Epoch 9, NEG ELBO: 35.574
Epoch 10, NEG ELBO: 41.046
set lr to 1.000e-02
Epoch 11, NEG ELBO: 39.205, Slope: -0.035
set lr to 1.000e-02
Epoch 12, NEG ELBO: 45.452, Slope: -0.017
Epoch 13, NEG ELBO: 42.442, Slope: -0.003
decrease lr to 9.000e-03
Epoch 14, NEG ELBO: 42.825, Slope: 0.009
decrease lr to 8.100e-03
Epoch 15, NEG ELBO: 39.589, Slope: 0.013
decrease lr to 7.290e-03
Epoch 16, NEG ELBO: 35.672, Slope: 0.009
decrease lr to 6.561e-03
Epoch 17, NEG ELBO: 36.396, Slope: 0.003
Epoch 18, NEG ELBO: 37.356, Slope: -0.007
set lr to 7.217e-03
Epoch 19, NEG ELBO: 39.774, Slope: -0.012
set lr to 7.939e-03
Epoch 20, NEG ELBO: 38.354, Slope: -0.013
set lr to 8.733e-03
Epoch 21, NEG ELBO: 37.539, Slope: -0.016
Epoch 22, NEG ELBO: 41.198, Slope: -0.006
decrease lr to 7

...

100%|████████████████████████████████████████| 100/100 [00:00<00:00, 258.44it/s]


Log evidence: -154.490
Updating hyperparameters...


100%|██████████████████████████████████████████| 50/50 [00:00<00:00, 286.50it/s]


Updating posterior...
Epoch 1, NEG ELBO: 82.965
Epoch 2, NEG ELBO: 62.438
Epoch 3, NEG ELBO: 63.756
Epoch 4, NEG ELBO: 59.759
Epoch 5, NEG ELBO: 63.475
Epoch 6, NEG ELBO: 65.228
Epoch 7, NEG ELBO: 64.671
Epoch 8, NEG ELBO: 60.684
Epoch 9, NEG ELBO: 64.576
Epoch 10, NEG ELBO: 62.781
decrease lr to 9.000e-03
Epoch 11, NEG ELBO: 79.476, Slope: 0.012
decrease lr to 8.100e-03
Epoch 12, NEG ELBO: 80.322, Slope: 0.021
decrease lr to 7.290e-03
Epoch 13, NEG ELBO: 68.756, Slope: 0.020
decrease lr to 6.561e-03
Epoch 14, NEG ELBO: 66.923, Slope: 0.014
decrease lr to 5.905e-03
Epoch 15, NEG ELBO: 67.494, Slope: 0.011
decrease lr to 5.314e-03
Epoch 16, NEG ELBO: 63.624, Slope: 0.005
Epoch 17, NEG ELBO: 61.501, Slope: -0.002
Epoch 18, NEG ELBO: 64.321, Slope: -0.010
set lr to 5.846e-03
Epoch 19, NEG ELBO: 60.358, Slope: -0.018
set lr to 6.430e-03
Epoch 20, NEG ELBO: 64.025, Slope: -0.024
set lr to 7.073e-03
Epoch 21, NEG ELBO: 68.220, Slope: -0.014
Epoch 22, NEG ELBO: 58.980, Slope: -0.009
Epoch 23,

  0%|                                                   | 0/100 [00:00<?, ?it/s]